In [0]:
import os
import sys
import numpy as np
import numba as nb
import numba.cuda as cuda    

def check_gpu():
    print("checking gpu")
    A = np.arange(3)
    A_gpu = cuda.to_device(A)
    @cuda.jit
    def double_gpu(A):
        tx = cuda.threadIdx.x
        A[tx] = 2*A[tx]
    double_gpu[1,3](A_gpu)
    return True

gpu_ok = False
try:
    gpu_ok = check_gpu()
except cuda.CudaSupportError:
    sys.exit("GPU not detected - Are you sure you have a GPU?  If using Colab, Runtime->Change Runtime Type->Hardware accelerator = GPU")
except cuda.cudadrv.nvvm.NvvmSupportError:
    fix = input("NvvmSupportError - either you do not have cudatoolkit or you don't have the environ set.  Would you like me to try to fix it (y/n)?").lower()
    if fix == "y" or fix == "yes":
        try:
            print("Setting environment")
            os.environ['NUMBAPRO_LIBDEVICE'] = "/usr/lib/nvidia-cuda-toolkit/libdevice"
            os.environ['NUMBAPRO_NVVM'] = "/usr/lib/x86_64-linux-gnu/libnvvm.so"
            gpu_ok = check_gpu()
        except cuda.cudadrv.nvvm.NvvmSupportError:
            while not gpu_ok:
                try:
                    install_mode = input("I think you need to install cudatoolkit; perhaps I can help.  Would you like me to (c) conda install, (p) pip install, or (n) do nothing?").lower()
                    if install_mode == "c":
                        os.system('conda install -c conda-forge cudatoolkit')
                    elif install_mode == "p":
                        os.system('apt-get update')
                        os.system('apt install -y --no-install-recommends -q nvidia-cuda-toolkit')
                        os.system('apt-get update')
                    else:
                        break
                    os.environ['NUMBAPRO_LIBDEVICE'] = "/usr/lib/nvidia-cuda-toolkit/libdevice"
                    os.environ['NUMBAPRO_NVVM'] = "/usr/lib/x86_64-linux-gnu/libnvvm.so"
                    gpu_ok = check_gpu()
                except:
                    pass

if not gpu_ok:
    sys.exit("GPU check failed - exiting")
    


checking gpu
